# Vertex-Cover

<p>Aqui é apresentado uma modelagem para o entendimento da prova de que VERTEX-COVER é NP completo através da redução para 3SAT. O 3SAT é um problema NP completo, então, se reduzirmos ele a outro problema, A, com um redutor em tempo polinomial, segue-se que A é NP completo também.</p>

$A {<=}_{p} 3SAT => A \in NPCompleto$

### Setup

In [12]:
def create_graph(vertices):
    G = {}
    G["vertices"] = vertices
    vertices_copy = vertices.copy()

    edges_set = {}
    for i in G['vertices'].keys():

        for j in vertices_copy[i]:
            
            if i == j:
                continue

            connection = vertices_copy.get(j, None)
            if connection != None:
                edges_set[(i,j)] = False

        vertices_copy.pop(i)

    G["edges"] = edges_set
    return G

## Vertex Verification algorithm

In [21]:
def verifier(G, v_set_subgraph):

    if len(G['vertices']) < len(v_set_subgraph):
        return False

    for v in v_set_subgraph: #len(v_set_subgraph) = k <= n
        vertex = G['vertices'].get(v, None)
        
        if vertex == None:
            return False

        for i in vertex: #len(vertex) <= n-1

            if G["edges"].get((v,i),None) != None: #O(1)
                G["edges"].pop((v,i)) #O(1)
            elif G["edges"].get((i,v),None) != None: #O(1)
                G["edges"].pop((i,v)) #O(1)

    if len(G["edges"]) > 0:
        return False
    else:
        return True

## Test with an example

In [23]:
sub_graph = {1,2,4,3,6}

G = create_graph({1: {2,3}, 2:{1,3,4,5}, 3:{1,2,4,6}, 4:{3,7,2,5},5:{2,4},6:{3,7},7:{6,4}})

verifier(G, sub_graph)

True

## Reduction to 3SAT

**Shape of 3SAT**

Ex.: $\phi = (x_1 \lor x_1 \lor x_2) \land (\bar{x_1} \lor \bar{x_2} \lor \bar{x_2}) \land (\bar{x_1} \lor x_2 \lor x_2)$

In [8]:
def reduction_3SAT_to_vertex(variable_set, phi):
    vertices = {}

    for x in variable_set.keys(): #n nodes
        vertices[x] = {x+1} #x and ~x

    for clause in phi:
        
        #connect the first with each other
        if vertices.get(clause[0], None) == None:
            vertices[clause[0]] = set()
            
        vertices[clause[0]].add(clause[1])
        vertices[clause[0]].add(clause[2])

        if vertices.get(clause[1], None) == None:
            vertices[clause[1]] = set()
        
        vertices[clause[1]].add(clause[2])

    return create_graph(vertices)


**Test with an example**

Ex.: $\phi = (x_1 \lor x_1 \lor x_2) \land (\bar{x_1} \lor \bar{x_2} \lor \bar{x_2}) \land (\bar{x_1} \lor x_2 \lor x_2)$

with variable set = **{1:"x1",3:"x2"}**

the equation is defined as **phi = {(1,1,3),(2,4,4),(2,3,3)}**

where **{1 -> "x1",2 -> "~x1",3 -> "x2",4 -> "~x2"}**

In [25]:
variable_set = {1:"x1",3:"x2"} #define a set of variables. x must be in odd position like 1,3,5,7...

phi = {(1,1,3),(2,4,4),(2,3,3)}

G = reduction_3SAT_to_vertex(variable_set, phi)

sub_graph = {3,2} #Possible solution: {x2,~x1} = {3,2}
print("{x2,~x1} is solution: " + str(verifier(G, sub_graph)))

sub_graph = {1,4} #Not solution: {x1, ~x2}
print("{x1, ~x2} is solution: " + str(verifier(G, sub_graph)))

{x2,~x1} is solution: False
{x1, ~x2} is solution: True
